In [2]:
import numpy as np

In [4]:
def reconstruct(W, H):
    N = W.shape[0]
    K = W.shape[1]
    L = W.shape[2]
    T = H.shape[1]
    
    H = np.hstack(np.zeros([K, L]), H, np.zeros([K, L]))
    T += 2*L
    X_hat = np.zeros([N, T])
    
    for t in np.arange(L):
        X_hat += W[:, :, t] * np.roll(H, t-1, axis=1)
    
    return X_hat[:, L+1:-L]

In [ ]:
def seq_nmf(X, K=10, L=20, lambda=.1, W_init=None, H_init=None,...
            plot_it=True, maxiter=20, tol=-np.inf, shift=1, sort_factors=1, ...
            lambda_L1W=0, lambda_L1H=0, lambda_OrthH=0, lambda_OrthW=0, M=None, ...
            use_W_update=True):
    N = X.shape[0]
    T = X.shape[1]
    if W_init is None:
        W_init = np.max(X) * np.random.rand([N, K, L])
    if H_init is None:
        H_init = np.max(X) * np.random.rand([K, T]) / np.sqrt(T / 3)
    if M is None:
        M = np.ones([N, T])
    
    assert np.all(X >= 0), 'all data values must be positive!'
    
    W = W_init
    H = H_init
    
    Xhat = reconstruct(W, H)
    mask = M == 0
    X(mask) = Xhat(mask)
    
    smooth_kernel = np.ones([1, (2*L) - 1])
    eps = np.max(X) * 1e-6
    last_time = 0
    
    cost = np.zeros([maxiter+1, 1])
    cost[0] = np.sqrt(np.mean(np.pow(X - Xhat, 2)))
    
    ##STOPPED AT LINE 106